# TP Pobabilités Numériques - Stats Computationnelles
## Paul LE FRANC - Emilio PICARD

## Inférence variationnelle

## Sujet : Application d'une approche variationnelle pour un modèle de régression bayésien

#### Modèle :

On considère un modèle de régression linéaire, de la forme $y = \beta^T x$, $y = (y_i)_{i=1,...,n} \in \mathbb{R}^n$, $x = (x_i)_{i=1,...,n} \in \mathbb{R}^n$x$\mathbb{R}^D,  x_i \in \mathbb{R}^D$.

La variable latente $\beta \in \mathbb{R}^D$ est le vecteur des coefficients de régression.

La loi de $y$ dépend donc de $\beta$ aléatoire, mais aussi de sa variance inconnue, car nous ferons l'hypothèse que les $y_i$ suivent une loi normale, les $y_i$ étant $iid$. On a donc: 
$$
p( y \vert \beta,\tau ; x) = \prod_{i=1}^{n} \phi_{\mathcal{N}(\beta^T x_i,\tau)}(y_i),
$$
où $\phi_{\mathcal{N}(\beta^T x_i,\tau)}$ est la densité d'une loi normale de moyenne $\beta^Tx_i$ et de paramètre de précision $\tau_i = \frac{1}{\sigma_i^2}, \sigma^2$ étant la variance, $\tau \in \mathbb{R}^n$.

Pour faciliter les notations, nous noterons $\phi_{\mathcal{N}(\mu,\tau)}(z) = \mathcal{N}(z \vert \mu,\tau).$

Le cadre de notre étude repose sur du Bayésien hiérarchique, c'est à dire que la loi a priori $\mathcal{L}(\beta,\tau)$ est conditionnée par rapport à une autre variable $\alpha$, elle même aléatoire suivant une certaine loi. On a donc :

$$
p(\beta,\tau \vert \alpha) = \mathcal{N}(\beta \vert 0, \tau diag (\alpha)) \Gamma(\tau \vert a_0, b_0),
$$
où $\alpha$ est un vecteur de dimension D, aléatoire, tel que :
$$
p(\alpha) = \prod_{d=1}^D \Gamma (\alpha_d \vert c_0, d_0),
$$
$a_0, b_0, c_0$ et $d_0$ fixés, appelés Hyper-paramètres.

#### Importation des bibliothèques qui vont être utilisées 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

#### Problème et Solution :

Notre objectif est d'approcher $p(\beta,\tau \vert y;x)$, densité a posteriori du modèle, car nous ne pouvons pas la calculer directement. Posons $z =(\beta,\tau)$.

On propose une famille de densités candidates pour approcher $(z,\alpha) \mapsto p(z,\alpha \vert y;x) $. On introduit alors $\mathcal{D} = \{ (z,\alpha) \mapsto q(z,\alpha) = q(z)q(\alpha) \} $, la famille de densités en question.

D'après le cours les densités $q_j$ optimales sont:
$$
q_j(z_j) \propto \exp \{ \mathbb{E}_{-j} [ \log p(z_j, z_{-j}, y,x) ] \},
$$
où les $z_j$ sont $z$ et $\alpha$.

#### Construction de l'algorithme de CAVI :

Commençons, graçe à la formule ci-dessus, par trouver la forme optimale de $q(z) = q(\beta, \tau)$:

\begin{align*}
\log p( \beta ,\tau, \alpha, x, y) &= \log p( y \vert \beta, \tau, \alpha ) + \log p(\beta, \tau, \alpha) \\
&= \log p(y \vert \beta, \tau, x ) + \log p( \beta,\tau \vert \alpha ) + \log p(\alpha)
\end{align*}

D'où :

\begin{align}
\log q^*(\beta, \tau) &= \mathbb{E}_{\alpha} [ \log p(y \vert \beta, \tau, x) + \log p(\beta, \tau \vert \alpha)] + cste \\
&= \log p(y \vert \beta, \tau, x) + \mathbb{E}_{\alpha} [\log p(\beta, \tau \vert \alpha)] + cste \\
&= \log \{ \prod_i \mathcal{N}(y_i \vert \beta^T x_i,\tau) \} + \mathbb{E}_{\alpha} [\log p(\beta, \tau \vert \alpha)] + cste
\end{align}

Calculons d'abord $\log \{ \prod_i \mathcal{N}(y_i \vert \beta^T x_i,\tau) \}$:

\begin{align}
\log \{ \prod_i \mathcal{N}(y_i \vert \beta^T x_i,\tau) \} &= \sum_i \log \frac{\tau^{1/2}}{\sqrt{2\pi}} \exp ( -\frac{\tau}{2} (y_i - \beta^Tx_i)^2) \\
&= \frac{n}{2} \log \tau - \frac{\tau}{2} \sum_i (y_i - \beta^T x_i)^2 + cste \\ 
&= \frac{n}{2} \log \tau - \frac{\tau}{2} \sum_i y_i^2 + \frac{\tau}{2}2\beta^T \sum_i x_i y_i - \frac{\tau}{2} \beta^T \sum_i x_i x_i^T \beta + cste. 
\end{align}

Continuons avec le terme $\mathbb{E}_{\alpha} [\log p(\beta, \tau \vert \alpha)]$ :

\begin{align}
\mathbb{E}_{\alpha} [\log p(\beta, \tau \vert \alpha)] \propto \mathbb{E}_{\alpha} [\frac{D}{2} \log \tau - \frac{1}{2} \beta^T \tau diag(\alpha) \beta - b_0 \tau + (a_0 -1)\log\tau] \\
\propto \frac{D}{2} \log \tau - \frac{\beta^T}{2} \tau \mathbb{E}_{\alpha}[diag(\alpha)] \beta - b_0 \tau + (a_0 - 1) \log \tau.
\end{align}

Finalement, on obtient:

\begin{align}
\log q^* (\beta, \tau) \propto \log \tau (\frac{D}{2} + a_0 - 1 + \frac{n}{2}) - \frac{\tau}{2} ( \sum_i y_i^2 - 2 \beta^T \sum_ix_iy_i + \beta^T\sum_ix_ix_i^T\beta^T + 2b_0 + \beta^T \mathbb{E}_{\alpha} [diag(\alpha)]\beta ). 
\end{align}

Et en posant

\begin{align}
V_*^{-1} &= \mathbb{E}_{\alpha}[diag(\alpha)] + \sum_ix_ix_i^T \, , \\
\beta_* &= V_*\sum_ix_iy_i \,, \\
a_* &= a_0 + \frac{n}{2} \,, \\
b_* &= b_0 \frac{1}{2} ( \sum_iy_i^2 - \beta_*^T V_*^{-1} \beta_* )
\end{align}

on arrive à nos fins :

$$
\log q^*(\beta, \tau) \propto \log \mathcal{N}(\beta \vert \beta_*, \tau V_*^{-1}) + \log \Gamma( \tau \vert a_*, b_* ),
$$
ie
<font color=darkred>
$$
\boxed{q^*(\beta, \tau) \propto \mathcal{N}(\beta \vert \beta_*, \tau V_*^{-1}) \Gamma( \tau \vert a_*, b_* )}.
$$
</font>

Grâce à des calculs similaires, nous pouvons donner la forme optimale de $q(\tau)$.

On pose $$ q(\alpha) = \prod_{d=1}^D q(\alpha_d). $$

On trouve alors : 

<font color=darkred>
$$
\boxed{q^*(\alpha) \propto \Gamma( \alpha_d \vert c_*, d_{*d} )},
$$
</font>
avec $c_* = c_0 + \frac{D}{2}$  et   $d_{*d} = d_0 + \frac{1}{2}\mathbb{E}_{\beta,\tau}[\tau \beta_d^2]$.

Pour finir, afin de pouvoir coder l'algorithme CAVI, nous implémenterons les Espérances par :
\begin{align}
\mathbb{E}_{\alpha}[diag(\alpha)] &= c_* diag(1/d_*) \,, \\
\mathbb{E}_{\beta,\tau}[\tau\beta_d^2] &= \beta_{*d}^2a_*/b_* + [V_*]_d,
\end{align}
où $[.]_d$ est la d-ième valeur sur la diagonale d'une matrice.

#### Mise à jour de $\beta$ et $\tau$ avec CAVI :

In [144]:
X = np.array([[1,2,3],[2,2,2],[7,1,3],[1,1,2]])
Y = np.array([1,2,3,1])
d = np.array([1,1,1])
a = 2
b = 1
V_ =np.array([[3,2,1],[2,1,1],[1,1,1]])
beta = np.array([2,3,2])
c=5

In [145]:
def CAVI_update_V_(X,c,d):
    """
    Inputs :
    -------
    X same as above
    c : previous value, real
    d : previous value, D-dimensional vector
    
    Outputs :
    -------
    V_ : new estimation of V_, real
    
    """
    
    return c*np.diag(1/d) + X.T@X

In [146]:
def CAVI_update_beta(X,Y,V_): #renvoie la mise à jour de beta
    """
    Inputs :
    -----
    X n.D-dimensional matrix
    Y : data, n-dimensional vector
    V_ : (D,D)-dimensional matrix, new estimation of V_
    
    Outputs :
    -----
    beta : D-dimensional vector, new estimation of beta
    """
    X = np.array(X)
    Y = np.array(Y)
    
    num = X.T@Y
    
    return np.linalg.inv(V_)@num 


#denom = np.sum(np.diag(X@X.T)) + c*np.diag(1/d)    

In [171]:
def CAVI_update_a_b(Y,a,b,beta,V_): #renvoit la mise à jour des paramètres pour tau
    
    """
    Inputs :
    -----
    X, Y, V_ same as algorithm above 
    a : current estimation of a, real
    b : current estimation of b, real
    beta : new estimation of beta
    
    Outputs :
    ------
    a : new estimation of a, 1-dimensional
    b : new estimation of b, 1-dimensional vector
    
    """
    
    a_new = a + Y.shape[0]/2
    
    b_new = b + 1/2*(np.sum(Y**2) - (beta.T @ V_ @ beta).ravel() )
    
    
    return a_new, b_new[0]

#### Mise à jour des $\alpha$ avec CAVI :

In [163]:
def CAVI_update_c_d(X, Y, c, d, beta, a, b, V_):
    """
    Inputs :
    --------
    X,Y as above
    c, d, a, b, V_, beta : current estimation, before the CAVI_update
    
    Outputs :
    ------
    c : new estimation of c, real
    d : new estimation of d, D-dimensional vector
    
    """
    
    D = beta.shape[0]
    
    c_new = c + D/2  # ici nous avons choisi de mettre D/2 au lieu de 1/2
    d_new = d + 1/2*(beta**2*a/b + np.diag(np.linalg.inv(V_)))
    
    return c_new, d_new

In [172]:
CAVI_update_a_b(Y,a,b,beta,V_)

(4.0, -26.0)

Maintenant que toutes les mises à jours sont implémentées, il ne reste plus qu'à implémenter un algo comprenant toutes les mises à jours :

In [174]:
def CAVI_complet(X, Y, a0, b0, c0, d0, beta, V_, max_iter = 500, eps = 1e-8):
    """
    Inputs :
    -------
    a, b, c, d : Hyper-parameters of our model
    beta : D-dimensional vector, initial value
    V_ : (D,D)-dimentional matrix, initial estimation value
    X : (n,D)-dimentional matrix,
    Y : data, n-dimensional vector
    
    Outputs:
    -------
    ELBO, a, b, c, d, beta, V_ : all the value for each iteration
    
    """
    
    iter = 0
    
    n = Y.shape[0]
    D = beta.shape[0]
    
    A, B, C, D, Beta = [], [], [], [], []
    
    V_ = CAVI_update_V_(X, c0, d0)
    
    beta = CAVI_update_beta(X, Y, V_)
    Beta.append(beta)
    
    a, b = CAVI_update_a_b(Y, a0, b0, beta, V_)
    A.append(a)
    B.append(b)
    
    c, d = CAVI_update_c_d(X, Y, c0, d0, beta, a0, b0, V_)
    C.append(c)
    D.append(d)

    while iter <= max_iter or ELBO > eps :
        
        V_ = CAVI_update_V_(X, c, d)
    
        beta = CAVI_update_beta(X, Y, V_)
        Beta.append(beta)
    
        a, b = CAVI_update_a_b(Y, a, b, beta, V_)
        A.append(a)
        B.append(b)
    
        c, d = CAVI_update_c_d(X, Y, c, d, beta, a, b, V_)
        C.append(c)
        D.append(d)

        iter += 1
    
    return A, B, C, D, 

Pour la ELBO, après beaucoup de questionnement sur la manière de caluculer $\mathbb{E}[\log X]$, $X$ suit une $\Gamma(a,b)$ nous avons trouvé une alternative, et allons utiliser la fonction di-gamma, définie comme suit :
$$
\psi(z) = \frac{\Gamma(z)^{'}}{\Gamma(z)}
$$



In [180]:
from scipy.special import psi